In [1]:
#save the json data into text file
from flask import Flask
from flask import request
import requests
import json
from flask import render_template
import psycopg2
import time
from datetime import datetime


In [2]:
#first close the localhost window then interrupt kernel
app = Flask(__name__)
@app.route('/', methods=['GET','POST'])
def hello():
    str="Welcome to Data visualization,Current cryptocurrencies under consideration are \n 1)Bitcoin-BTC \n 2)Ethereum-ETH \n 3)Litecoin-LTC \n 4)BitcoinCash-BCH \n 5)Ripple-XRP\n"
    return str


In [3]:
@app.route('/cryptoPrice',methods=['GET','POST'])
def cryptoPrice():
    cryptoS=raw_input()
    cryptoT=raw_input()
    url = "https://min-api.cryptocompare.com/data/price?fsym={}&tsyms={}".format(cryptoS,cryptoT)
    try:
        response=requests.get(url)
        return response.text
    except:
        return "Connection Error"
#if (__name__ == "__main__"):
#    app.run(debug=False)

#Data Visualization through Graphs

In [4]:
import pygal
import json
from datetime import datetime
import time


#Graph for a specific Cryptocurrency

In [5]:
#to access say,BTC,over duration of day: URL http://127.0.0.1:5000/BTC/day/
#to access say,XRP,over duration of month : URL http://127.0.0.1:5000/XRP/month/

@app.route("/individual/<cryptoname>/<duration>/",methods=['GET','POST'])
def IndividualGraph(cryptoname,duration):
    cur.execute("select symbol,price_usd,last_updated from public.currency where symbol='{}'".format(cryptoname)+";")
    rows=cur.fetchone()
    #30 minutes time gap per entry for a day:48 rows
    if(duration=='day'):
        cur.execute("select symbol,price_usd,last_updated from public.currency where symbol='{}'".format(cryptoname)+";")
        rows=cur.fetchmany(48)
    elif(duration=='month'):
        #30 minute time gap entry for a month : 1440 rows,currently since that many rows arent available ,700 rows 
        cur.execute("select symbol,price_usd,last_updated from public.currency where symbol='{}'".format(cryptoname),";")
        rows=cur.fetchmany(700)
        
    elif(duration=='year'):
        cur.execute("select symbol,price_usd,last_updated from public.currency where symbol='BTC';")
        rows=cur.fetchall()
#plotting interactive svg
    cryptod_x_axis_data=[]
    cryptod_y_axis_data=[]
    for i in range(len(rows)):
        tupl= rows[i]
        var= tupl[2]
        if type(var)==float:
            cryptod_x_axis_data.append((datetime.fromtimestamp(tupl[2])).strftime('%Y-%m-%d %H:%M:%S'))
            cryptod_y_axis_data.append(tupl[1])
    #conn.close()
    chart=pygal.Line(x_label_rotation=45,x_labels_major_every=10,show_minor_x_labels=False)
    chart.x_labels=cryptod_x_axis_data
    chart.add(cryptoname+" "+duration,cryptod_y_axis_data)
    #if you want to download graph uncomment below
    chart.render_to_file('static/images/'+cryptoname+duration+'.svg')
    chart=chart.render_data_uri()
    
    return render_template( 'charts.html', chart = chart)
    
        

    

In [6]:
#Comparison graph of all 5 cryptocurrency

In [7]:
#to find a graph comparing 5 Cryptocurrency over duration of day/month url: http://127.0.0.1:5000/AllGraphs/day/ 
#or url: http://127.0.0.1:5000/AllGraphs/month/
@app.route("/AllGraphs/<duration>/",methods=['GET','POST'])
def LineGraph(duration):
    cur.execute("select symbol,price_usd,last_updated from public.currency where symbol='BTC';")
    rows=cur.fetchone()
    coins=['BTC','ETH','LTC','XRP','BCH']
    chart=pygal.Line(x_label_rotation=45,x_labels_major_every=10,show_minor_x_labels=False)
    for cryptoname in coins:
        if(duration=='day'):
            
            cur.execute("select symbol,price_usd,last_updated from public.currency where symbol='{}'".format(cryptoname)+";")
            rows=cur.fetchmany(48)
        elif(duration=='month'):
            cur.execute("select symbol,price_usd,last_updated from public.currency where symbol='{}'".format(cryptoname),";")
            rows=cur.fetchmany(700)       
        cryptod_y_axis_data=[]
        for i in range(len(rows)):
            tupl= rows[i]
            var= tupl[2]
            if type(var)==float:
                cryptod_y_axis_data.append(tupl[1])
            
        chart.add(cryptoname+" "+duration,cryptod_y_axis_data)
    cryptod_x_axis_data=[]
    
    for i in range(len(rows)):
        tupl=rows[i]
        var=tupl[2]
        if type(var)==float:
            cryptod_x_axis_data.append((datetime.fromtimestamp(tupl[2])).strftime('%Y-%m-%d %H:%M:%S'))
    chart.x_labels=cryptod_x_axis_data
    #uncomment next line if you want to save graph in file 
    chart.render_to_file('static/images/All'+duration+'.svg')
    #uncomment next 2 lines if graph response embedded in html is needed
    #chart=chart.render_data_uri()
    #return render_template( 'charts.html', chart = chart)
    return chart.render_response()
    
        


In [ ]:
if (__name__ == "__main__"):
    #establishing connections
    hostname = 'baasu.db.elephantsql.com'
    username = 'dbuzkqmi'
    password = 'vi24qSFc5TG77k5GPa4aQr3XlnLOBIRf'
    database = 'dbuzkqmi'
    port='5432'
    conn = psycopg2.connect(host=hostname, user=username, password=password, dbname=database,port=port)
    #conn=connection()
    curr=conn.cursor()
    password ='12345'
    email='goyalakshita@gmail.com'
    name='Akshita'
    curr.execute("insert into public.user (password,email,name) values('{}','{}','{}')".format(password,email,name))
    cur=conn.cursor()
    app.run(debug=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
